In [48]:
import pandas as pd
import numpy as np
import re
import json
from nltk.corpus import stopwords

In [49]:
stopword = stopwords.words('english')

In [50]:
# df = pd.read_excel('2020may30_tw_FindMiss.xlsx')
df = pd.read_csv('2020july21_FB1500_FindMiss.csv') # '2020july21_FB1500_FindMiss.csv' is made by combining test datasets

In [51]:
df.head()

,Gold Label,words,m_label,covid,economic,education,environmental,foreign policy,governance,health,immigration,military,safety,social cultural,social programs
0,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
1,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
2,economic,"financial protection,banks,economy,consumers,p...",economic,0,correct label,0,0,0,0,0,0,0,0,0,0
3,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
4,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
df2=df.loc[(df['economic']!='0')]

df2.shape

(181, 15)

In [53]:

def filter_lexicon(lexicon):
    topics= lexicon.keys()
    for t in topics:
        ls=[]
        for phrase in lexicon[t]:
            # phrase = ' '.join([w.lower() for w in phrase.split() if w.lower() not in stopword])
            phrase = ' '.join([w.lower() for w in phrase.split()])
            ls.append(phrase)            
        lexicon[t]=ls
    return lexicon

In [55]:
# with open ('delete_dict.json','r')as f1:
#     dic = json.load(f1)

with open ('remove_V461.json','r')as f1:
    dic = json.load(f1)
    
dic1 = filter_lexicon(dic)

In [56]:
sorted(dic1)

['covid',
 'economic',
 'education',
 'environmental',
 'foreign policy',
 'governance',
 'health',
 'immigration',
 'military',
 'safety',
 'social cultural',
 'social programs']

In [57]:
def create_table(df,dic,x):
    length = len(df.loc[(df[x]!='0')])
    # print(length)
    df1= df.loc[(df[x]!='0')]
    df1 = df1[['Gold Label','words',x]]
    words = dic[x]
    sdf = pd.DataFrame({k:np.zeros(length,dtype=int)for k in words})
    ndf = df1.join(sdf,lsuffix='_caller')
    # ndf = ndf[ndf['words'].notna()]
    ndf = ndf[ndf.words != 'no words']
    ndf.fillna('0',inplace=True)
    return ndf

In [58]:
def caculate_table (df,dic,x):
    ndf = create_table(df,dic,x)
    
    for index,row in ndf.iterrows(): 
        try:
            if row[x]=='correct label':
                ndf.loc[index,3:]='na' # which means not appear
                l1 = row['words'].split(',')
                l2 = dic[x]
                ls = list(set(l1)&set(l2))
                for w in ls:
                    ndf.loc[index,w]= 1  
#                 col = tuple(set(l2).difference(set(ls)))
                
                
            if row[x]!='correct label':
                ndf.loc[index,3:]='na'
                ls = row[x].split(',')
                # col = tuple(set(dic[x]).difference(set(ls)))
                col = ls
                ndf.loc[index,col]= 0
                     
                   
        except KeyError:
            # print('keyError')
            if row[x+'_caller']=='correct label':
                ndf.loc[index,3:]='na' # which means not appear
                l1 = row['words'].split(',')
                l2 = dic[x]
                ls = list(set(l1)&set(l2))
                for w in ls:
                    ndf.loc[index,w]= 1  
#                 col = tuple(set(l2).difference(set(ls)))
                                
            if row[x+'_caller']!='correct label':
                ndf.loc[index,3:]='na'
                ls = row[x+"_caller"].split(',')
                # col = tuple(set(dic[x]).difference(set(ls)))
                col = ls
                ndf.loc[index,col]= 0
                            
    ndf.dropna(how='all',axis=0,inplace=True)
    
    return ndf

In [59]:
sdf = create_table(df,dic1,'social programs')
sdf

,Gold Label,words,social programs,affordable healthcare,public schools,diseases,mental health,affordable health care,getting health,hbcus,...,health care,academic,education plan,student,diagnosis,welfare,affordable child,debt,high quality,student loan debt
14,education,"student,college,debt","college,student,debt",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,social programs,social security,correct label,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,social programs,social security,correct label,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,"health,social programs,economic","middle class,health care,health,healthcare",correct label,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,"economic,governance,health,environmental","health insurance,wealth tax,economic,tax,healt...",health insurance,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,"health,social programs","medicare,medicare for all",correct label,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1449,"social programs,health","health insurance,medicare,health,medicare for all",correct label,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1466,"education,health,governance","affordable healthcare,education,healthcare",affordable healthcare,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1482,social cultural,welfare,welfare,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
topics=dic1.keys()

table_list = []

for t in topics:
    sdf = caculate_table(df,dic1,t)
    table_list.append(sdf)

In [61]:
len(table_list)

12

In [62]:
table_list[-1]# ['health insurance']

,Gold Label,words,social programs,affordable healthcare,public schools,diseases,mental health,affordable health care,getting health,hbcus,...,health care,academic,education plan,student,diagnosis,welfare,affordable child,debt,high quality,student loan debt
14,education,"student,college,debt","college,student,debt",na,na,na,na,na,na,na,...,na,na,na,0,na,na,na,0,na,na
18,social programs,social security,correct label,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
19,social programs,social security,correct label,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
26,"health,social programs,economic","middle class,health care,health,healthcare",correct label,na,na,na,na,na,na,na,...,1,na,na,na,na,na,na,na,na,na
27,"economic,governance,health,environmental","health insurance,wealth tax,economic,tax,healt...",health insurance,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,"health,social programs","medicare,medicare for all",correct label,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
1449,"social programs,health","health insurance,medicare,health,medicare for all",correct label,na,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
1466,"education,health,governance","affordable healthcare,education,healthcare",affordable healthcare,0,na,na,na,na,na,na,...,na,na,na,na,na,na,na,na,na,na
1482,social cultural,welfare,welfare,na,na,na,na,na,na,na,...,na,na,na,na,na,0,na,na,na,na


In [63]:
with pd.ExcelWriter('july21caculate_wordtagging_rate.xlsx') as writer:
    for sdf,name in zip(table_list,topics):
        sdf.to_excel(writer, sheet_name=name)

In [64]:
def caculate_rate(df,dic,x):
    n_dic = {}
    words = dic[x]
    for w in words:
        if len(df.loc[(df[w]!='na')])!=0:
            correct = len(df.loc[(df[w]==1)])/len(df.loc[(df[w]!='na')])
            d={w:correct}
            n_dic.update(d)
        if len(df.loc[(df[w]!='na')])==0:
            d = {w:'???'}
            print(d)
    return {x:n_dic}

In [65]:
result = {}

for df,name in zip(table_list,topics):
    r_dict = caculate_rate(df,dic1,name)
    result.update(r_dict)

In [66]:
result

{'covid': {'covid': 0.875, 'covid 19': 0.875},
 'economic': {'middle class': 0.7,
  'ceos': 0.8,
  'jobs': 0.9090909090909091,
  'businessman': 0.7777777777777778,
  'wealth': 0.8571428571428571,
  'american worker': 0.25,
  'workers': 0.8148148148148148,
  'prosperity': 0.5,
  'american workers': 0.6666666666666666,
  'business leaders': 0.0,
  'treasurer': 0.0,
  'economy': 0.9117647058823529},
 'education': {'school': 0.8095238095238095,
  'education': 0.9629629629629629,
  'public education': 0.6666666666666666,
  'student': 0.6363636363636364,
  'public schools': 0.5,
  'loans': 0.6666666666666666,
  'teachers': 0.875},
 'environmental': {'climate': 0.9833333333333333,
  'environment': 0.8333333333333334,
  'climate change': 0.967741935483871,
  'ethanol': 0.0},
 'foreign policy': {'muslim ban': 0.6666666666666666,
  'foreign': 0.8823529411764706,
  'alliances': 0.0,
  'south korea': 0.0,
  'usmca': 0.8571428571428571,
  'aipac': 0.0,
  'israel': 0.0,
  'foreign money': 0.0},
 'go

In [67]:
with open ('lexiconV461_deletewordsTagRate.json','w') as f1:
    json.dump(result,f1,sort_keys=True,indent=6)